In [2]:
import time
import random as rd
import pandas as pd
import numpy as np
from tqdm import tqdm 

import requests # 페이지를 요청하는 기능 
from bs4 import BeautifulSoup # 파이썬으로 HTML을 다루는 기능
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

In [3]:
# 200: 정상 (가져오는데 충돌이 없었다., 400번대: 존재하지 않는 페이지)
# 500번대: 존재하지만 볼 수 없는 곳

url = "https://www.tripadvisor.com/Restaurants-g298085-Da_Nang.html"
response = requests.get(url, headers=headers)
print(response)
print(response.text)

<Response [200]>
<!DOCTYPE html><html lang="en-US"><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><meta http-equiv="content-language" content="en"/><link href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans-VF.woff2?v1.002" rel="preload" as="font" type="font/woff2" crossorigin="anonymous"/><link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico?v2" type="image/x-icon"/><link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" color="#000000"/><link rel="icon" sizes="192x192" type="image/png" href="https://static.tacdn.com/img2/brand_refresh/application_icons/touch-icon-192x192.png"/><meta name="theme-color" content="#34e0a1"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/>

In [4]:
response.raise_for_status()  # 200번대 코드가 아니면 코드가 멈춤
print("response successful!")

html = response.text
soup = BeautifulSoup(html, "html.parser")
# print(soup.prettify())

# '''파일 생성 및 저장하기'''
# # with open('파일명', '옵션', 인코딩) as f
# # 경로를 지정하지않으면 해당 디렉토리에 생성
# with open('test.html', 'w', encoding ='utf-8') as f:
#     f.write(response.text)

response successful!


In [5]:
dataset = pd.DataFrame({
    "name": [],
    "category": [],
    "description": [],
    "category": [],
    "opneingTime": [],
    "closingTime": [],
    "URL": [],
    "priceLow": [],
    "priceHigh": [],
    "reviews": [],
})

dataset

,name,category,description,opneingTime,closingTime,URL,priceLow,priceHigh,reviews


In [6]:
restraurant_pages = []
for restraurant_page in soup.find_all('div', class_="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"):
    restraurant_pages.append("https://www.tripadvisor.com/" + restraurant_page.find('a')['href'])

print(restraurant_pages)

['https://www.tripadvisor.com//Restaurant_Review-g298085-d17395398-Reviews-Veteran_Restaurant-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d24985405-Reviews-Cardi_Pizzeria_Bach_Dang-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d7000524-Reviews-Ca_Chuon_Co_Vietnamese_Seafood_Restaurant-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d13236259-Reviews-Bistecca_Restaurant_Danang-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d24082246-Reviews-3_Big_Nu_ng_L_u-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g15296807-d13810289-Reviews-Thia_G_Restaurant_Da_Nang-My_An_Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d14199145-Reviews-Cabanon_Palace-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d25073751-Reviews-La_Cabana_House_of_BBQ-Da_Nang.html', 'https://www.tripadvisor.com//Restaurant_Review-g298085-d3187207-Reviews-The_Golden_Dra

In [7]:
url = restraurant_pages[1]
response = requests.get(url, headers=headers)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")
# print(soup.prettify())

In [10]:
# name 
name = soup.find('h1', class_="HjBfq").text;    print(f'name: {name}')

# category
category = ''
for x in soup.find('span', class_="DsyBj DxyfE"):
    if '$' not in x.text:
        category += f'{x.text}, '
category = category[:-2]
print(f'category: {category}')

# adress
adress = soup.find('a', class_="AYHFM").text;   print(f'adress: {adress}')

# rating
rating = soup.find('span', class_="ZDEqb").text;    print(f'rating: {rating}')

# time
openingTime = soup.find('span', class_="mMkhr").text[11:19];    print(f'openingTime: {openingTime}')
closingTime = soup.find('span', class_="mMkhr").text[22:30];    print(f'closingTime: {closingTime}')

# url
url = url

# price
try:
    priceLow = soup.find('div', class_="SrqKb").text.split(' - ')[0] 
    priceHigh = soup.find('div', class_="SrqKb").text.split(' - ')[1]
except:
    priceLow = np.NaN; priceHigh = np.NaN
print (f'priceLow: {priceLow}, priceHigh: {priceHigh}')

# reviews, only english reviews
# page url
'''
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-or15-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-or30-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
'''
total_reviews = int(soup.find('span', class_="count").text[1:-1].replace(',', ''));    print(f'total_reviews: {total_reviews}')
review_pages = total_reviews // 15 if total_reviews%15 == 0 else total_reviews//15 + 1;    print(f'review_pages: {review_pages}')
for i in tqdm(range(1, review_pages, 2), desc='review_pages'):
    if i > 1:
        url_list = url.split('-Reviews')
        url_list.insert(1, f'-or{i*15}')
        current_url = ''.join(url_list)
        print(f'current_url: {current_url}')
        response = requests.get(current_url, headers=headers)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")

    reviews = soup.find_all('p', class_="partial_entry");
    for review in reviews:
        print(review.text)

    time.sleep(rd.uniform(1, 2))

dataset["name"] = [name]
dataset["category"] = [category]
dataset["adress"] = [adress]
dataset["rating"] = [rating]
dataset["opneingTime"] = [openingTime]
dataset["closingTime"] = [closingTime]
dataset["URL"] = [url]
dataset["priceLow"] = [priceLow]
dataset["priceHigh"] = [priceHigh]
dataset["reviews"] = [reviews]
dataset

name: Cardi Pizzeria Bach Dang
category: American, Steakhouse, Brew Pub
adress: #1 of 1,086 Restaurants in Da Nang
rating: 5.0 
openingTime: 12:00 AM
closingTime: 11:59 PM
priceLow: nan, priceHigh: nan
total_reviews: 333
review_pages: 23


review_pages:   0%|          | 0/11 [00:00<?, ?it/s]

Good food and drinks 🍹 
Pizza was incredible, my coconut milkshake was very tasty. 
Staff was so friendly and nice 😊
We tried the pizza, spaghetti, and salad at this restaurant. The pizza was delicious with its thin crust, while the spaghetti and salad were okay. Overall, worth a try.
Very nice corner setting. Great views of the river. The air-conditioning was a pleasant relief from the heat and humidity outside. 

Staff super nice and friendly. Smiles all around.

The pizza was delicious and just the right size for 1 person.  Prices very reasonable.
We popped in here after watching the fire breathing  Dragon on the bridge …  we ordered 2 pizzas, some chips, wine and beers .. the  pizzas are fresh, good portion of chips and they let me swap some of the ingredients over … the staff...More
This place serves delicious pizza and good beverage, friendly staff, great service and nice ambience.
Food is good, atmosphere is cozy.
I really enjoyed the meal. 

The waiters were nice and polite. 



review_pages:   9%|▉         | 1/11 [00:01<00:10,  1.05s/it]

current_url: https://www.tripadvisor.com//Restaurant_Review-g298085-d24985405-or45-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
Good food and drinks 🍹 
Pizza was incredible, my coconut milkshake was very tasty. 
Staff was so friendly and nice 😊
We tried the pizza, spaghetti, and salad at this restaurant. The pizza was delicious with its thin crust, while the spaghetti and salad were okay. Overall, worth a try.
Very nice corner setting. Great views of the river. The air-conditioning was a pleasant relief from the heat and humidity outside. 

Staff super nice and friendly. Smiles all around.

The pizza was delicious and just the right size for 1 person.  Prices very reasonable.
We popped in here after watching the fire breathing  Dragon on the bridge …  we ordered 2 pizzas, some chips, wine and beers .. the  pizzas are fresh, good portion of chips and they let me swap some of the ingredients over … the staff...More
This place serves delicious pizza and good beverage, friendly staff, great servi

...

current_url: https://www.tripadvisor.com//Restaurant_Review-g298085-d24985405-or285-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
As westerners we craved a bit of Italian. By far the best meal we had in Vietnam. The spaghetti with Raghu was excellent as well as the half and half pizza spinach rustico and prosciutto el fungi. Brilliant mix.
Outstanding service (My - the waitress served us with all her passion and enthusiasm), food is so good, best view at Han River, highly recommend for couple and family.
Amazing pizza, pumpkin soup was amazing and the garlic bread is a must try. 
There is vegetarian food too. If u r a vegetarian like my wife this a best place to fill her tummy with no complains
Stumbled across this little restaurant. Just what I needed for late night pizza! Very good and great service!
Came to dinner tonight for pasta and pizza, great food and even better service! A must if your in Da
We had a brilliant experience at Cardi! 

Food was amazing, very tasty and fresh. The service

review_pages:  91%|█████████ | 10/11 [00:39<00:04,  4.31s/it]

current_url: https://www.tripadvisor.com//Restaurant_Review-g298085-d24985405-or315-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
Best steak in the city
I was walking down the street and saw the restaurant, the menu, look so nice' so I decided go to try. I ordered the steak and and peroni beer it was so great. I love it here 
Highly recommend
Very good food,very good service,and good View:))! (Near Han River and near Han market:)) service very polite and very fresh and clean food:)) pizza very good same as Italian pizza :))!!
Great food and and really friendly staff. Ask for Anna for 5 star service! The four cheese pizza is superb. Great location over looking the river.
Food is good , I ordered a hawaian pizza. Service is friendly and staff speaks English. 
View from second floor to street is nice. Can enjoy sceneries while eating.
The place I can tell and suggest to all Thai Tourist to dine. They serve delicious pizza. A pizza that is perfect for couple, friends or family. They serve good dri

review_pages: 100%|██████████| 11/11 [00:42<00:00,  3.88s/it]


,name,category,description,opneingTime,closingTime,URL,priceLow,priceHigh,reviews,adress,rating
0,Cardi Pizzeria Bach Dang,"American, Steakhouse, Brew Pub",NaN,12:00 AM,11:59 PM,https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,[[Best steak in the city\nI was walking down t...,"#1 of 1,086 Restaurants in Da Nang",5.0


In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())
options = Options()

#지정한 user-agent로 설정합니다.
options.add_argument('user-agent=' + user_agent)
# options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
# options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
# options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
# options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
# options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
driver.set_window_size(1920, 1080)


# description
driver.get(url)
try:
    WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#component_52 > div.hILIJ > div > div:nth-child(2) > div > div > div.gmbZC > a")))
    element = driver.find_element(By.CSS_SELECTOR, "#component_52 > div.hILIJ > div > div:nth-child(2) > div > div > div.gmbZC > a")
    element.click()
except:
    driver.quit()
try:
    WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")))
    element = driver.find_element(By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")
    description = element.text
    print(description)
except:
    driver.quit()
print(f'description: {description}')

# reviews
reviews = []
try:
    WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.zPIck._Q.Z1.t._U.c._S.zXWgK")))
    element = driver.find_element(By.CSS_SELECTOR, "BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.zPIck._Q.Z1.t._U.c._S.zXWgK")
    element.click()
except:
    driver.quit()

# page url
'''
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-or15-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
https://www.tripadvisor.com/Restaurant_Review-g298085-d24985405-Reviews-or30-Cardi_Pizzeria_Bach_Dang-Da_Nang.html
'''

Premium Steak & Pizza Certified Imported Steak, Fresh Italian Standard Pizza, Premium Wine Cellar Millionaire Han River view. We have a chain of 4 restaurants in Da Nang and Hoi An: 124 Bạch Đằng, Đà Nẵng 76 Đường 2/9, Đà Nẵng, 93 Võ Văn Kiệt, Đà Nẵng and 32 Thái Phiên, Hội An
description: Premium Steak & Pizza Certified Imported Steak, Fresh Italian Standard Pizza, Premium Wine Cellar Millionaire Han River view. We have a chain of 4 restaurants in Da Nang and Hoi An: 124 Bạch Đằng, Đà Nẵng 76 Đường 2/9, Đà Nẵng, 93 Võ Văn Kiệt, Đà Nẵng and 32 Thái Phiên, Hội An


## Demo

In [2]:
def setRestaurantPages(url: str):
    url = url
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 200번대 코드가 아니면 코드가 멈춤
    print("response successful!")
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    restraurant_pages = []
    for restraurant_page in soup.find_all('div', class_="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"):
        restraurant_pages.append("https://www.tripadvisor.com/" + restraurant_page.find('a')['href'])
    print(restraurant_pages)
    
    return restraurant_pages

In [3]:
def createDataset():
    dataset = pd.DataFrame({
        "name": [],
        "category": [],
        "description": [],
        "category": [],
        "opneingTime": [],
        "closingTime": [],
        "URL": [],
        "priceLow": [],
        "priceHigh": [],
        "reviews": [],
    })
    
    return dataset

In [4]:
def setWebdriver():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    options = Options()
    options.add_argument('user-agent=' + user_agent)
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    # options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    # options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    # options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver.set_window_size(1920, 1080)

    return driver

In [12]:
def startCrawling(pages: list):
    for restaurant_page in pages:
        response = requests.get(restaurant_page, headers=headers)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")

        # name 
        name = soup.find('h1', class_="HjBfq").text;    print(f'name: {name}')

        # category
        category = ''
        for x in soup.find('span', class_="DsyBj DxyfE"):
            if '$' not in x.text:
                category += f'{x.text}, '
        category = category[:-2]
        print(f'category: {category}')

        # time
        openingTime = soup.find('span', class_="mMkhr").text[11:19];    print(f'openingTime: {openingTime}')
        closingTime = soup.find('span', class_="mMkhr").text[22:30];    print(f'closingTime: {closingTime}')

        url = restaurant_page;    print(f'url: {url}')

        # price
        try:
            priceLow = soup.find('div', class_="SrqKb").text.split(' - ')[0] 
            priceHigh = soup.find('div', class_="SrqKb").text.split(' - ')[1]
        except:
            priceLow = np.NaN; priceHigh = np.NaN
        print (f'priceLow: {priceLow}, priceHigh: {priceHigh}')

        # description
        driver.get(url)
        try:
            WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#component_52 > div.hILIJ > div > div:nth-child(2) > div > div > div.gmbZC > a")))
            element = driver.find_element(By.CSS_SELECTOR, "#component_52 > div.hILIJ > div > div:nth-child(2) > div > div > div.gmbZC > a")
            element.click()
            WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")))
            element = driver.find_element(By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")
            description = element.text
            print(description)
        except:
            description = np.NaN
        print(f'description: {description}\n')
        
        
        time.sleep(rd.uniform(0.1, 0.5))

    return 

In [1]:
import time
import random as rd
import pandas as pd
import numpy as np

import requests # 페이지를 요청하는 기능 
from bs4 import BeautifulSoup # 파이썬으로 HTML을 다루는 기능
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

# set restaurant pages
url = "https://www.tripadvisor.com/Restaurants-g298085-Da_Nang.html"
restraurant_pages = setRestaurantPages(url)

# create dataset
dataset = createDataset()

# set webdriver
driver = setWebdriver()

# start crawling
startCrawling(restraurant_pages)

NameError: name 'setRestaurantPages' is not defined